
Erstellung eines Bildklassifikators auf Basis des CIFAR-10 Dataset mit Keras (TensorFlow 2)


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from PIL import Image

In [3]:
import numpy as np
import urllib
import matplotlib

In [4]:
import math
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

tf.enable_eager_execution()

In [5]:
# Variablen für das Training
BATCH_SIZE = 64
EPOCHS = 50

In [6]:
# Für die 10 Klassen von CIFAR-10
CIFAR_10_CLASSES = ["Flugzeug","Auto","Vogel","Katze","Hirsch","Hund", "Frosch", "Pferd","Boot","LKW"]
NUM_CLASSES = 10

In [7]:
# Wir laden den Datenset über Keras
(images_train, labels_train), (images_test, labels_test) = cifar10.load_data()

In [8]:
# Test
plt.title(CIFAR_10_CLASSES[int(labels_train[25])])
plt.imshow(images_train[25])
plt.show()

In [9]:
images_train = np.array(images_train,dtype="float32")
images_test = np.array(images_test,dtype="float32")

In [10]:
images_train /= 255 # Damit die Werte zwischen 0 und 1 bleiben
images_test /=255

In [11]:
labels_train = to_categorical(labels_train, NUM_CLASSES)
labels_test = to_categorical(labels_test, NUM_CLASSES)

In [12]:
# Beispiel von einer Subclassing eines Keras-Modells
class MyCIFARModel(tf.keras.Model):
    def __init__(self):
        super(MyCIFARModel, self).__init__()
        self.conv_1 = Conv2D(32, (3, 3), padding='same',input_shape=(32, 32, 3),activation="relu")
        self.max_pool_1 = MaxPool2D(pool_size=(2,2))
        self.conv_2 = Conv2D(64, (3, 3), padding='same')
        self.max_pool_2 = MaxPool2D(pool_size=(2,2))
        self.conv_3 = Conv2D(64, (3, 3), padding='same')
        self.max_pool_3 = MaxPool2D(pool_size=(2,2))
        self.flatten = Flatten()
        self.dense_1 = Dense(512,activation="relu")
        self.softmax = Dense(10,activation='softmax')

    def call(self, inputs):

        x = self.conv_1(inputs)
        x = self.max_pool_1(x)
        x = self.conv_2(x)
        x = self.max_pool_2(x)
        x = self.conv_3(x)
        x = self.max_pool_3(x)
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.softmax(x)

        return x   

Variante 1: Definition des Modells mit Sequential

In [13]:
print("=== Variante 1 mit Sequential === ")

=== Variante 1 mit Sequential === 


In [14]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=(32, 32, 3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dense(NUM_CLASSES,activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics = ["accuracy"])
model.fit(images_train,labels_train, batch_size=BATCH_SIZE,epochs=EPOCHS)
scores = model.evaluate(images_test,labels_test)

Train on 50000 samples
Epoch 1/50
50000/50000 [==============================] - 49s 979us/sample - loss: 2.3065 - accuracy: 0.0944
Epoch 2/50
50000/50000 [==============================] - 49s 977us/sample - loss: 2.2825 - accuracy: 0.1559
Epoch 3/50
50000/50000 [==============================] - 49s 988us/sample - loss: 2.2649 - accuracy: 0.1834
Epoch 4/50
50000/50000 [==============================] - 59s 1ms/sample - loss: 2.2477 - accuracy: 0.2142
Epoch 5/50
50000/50000 [==============================] - 62s 1ms/sample - loss: 2.2286 - accuracy: 0.2349
Epoch 6/50
 1216/50000 [..............................] - ETA: 1:01 - loss: 2.2218 - accuracy: 0.2525

In [ ]:
print('Loss:', scores[0])
print('Accuracy:', scores[1])

In [ ]:
tf.saved_model.save(model,"cifar_model_sequential")
model.save("cifar_model.h5")
del model

In [ ]:
# Variante 2
print("=== Variante 2 mit Model-Subclassing === ")
model = MyCIFARModel()
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics = ["accuracy"])
model.fit(images_train,labels_train, batch_size=BATCH_SIZE,epochs=EPOCHS)
scores = model.evaluate(images_test,labels_test)
print('Loss:', scores[0])
print('Accuracy:', scores[1])

In [ ]:
# Wir speichern das Model mit tf.saved_model
model.save('cifar_model_subclassing', save_format='tf')

In [ ]:
# Erzeugt aktuell eine Fehlermeldung, wenn Modell-Subclassing
# model.save("cifar_model_subclassing.h5") 
# Lösung:
model.save_weights("cifar_model_subclassing_weights")
del model

In [ ]:
# Aus dem Google Colab von
# Francois Chollet:
# https://colab.research.google.com/drive/172D4jishSgE3N7AO6U2OKAA_0wNnrMOq#scrollTo=gMg87Tz01cxQ
my_new_model = MyCIFARModel()
my_new_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics = ["accuracy"])
my_new_model.load_weights("cifar_model_subclassing_weights")

In [ ]:
predictions = my_new_model.predict(images_train[25].reshape(-1, 32, 32, 3))
index_max_predictions = np.argmax(predictions)
print("Ergebnis: {}".format(CIFAR_10_CLASSES[index_max_predictions]))